---
title: "LOCF Development and Cluster Analysis"
ownership: "Tyler Technologies"
author: "Daniel J Fasteen, Ph.D."
topic: "AVM Analysis"
date: "2/28/2025"
format:
  html:
    embed-resources: true        # bundles Folium map
    smooth-scroll: true
    fontcolor: black
    toc: true
    toc-location: right
    toc-depth: 2
    code-fold: true    
    code-fold-show: false
resources: [locf_map.html]       # make sure the map ships with the doc
---

### Instantiate libraries

In [ ]:
#| echo: true
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
from branca.colormap import StepColormap

## Load data file from GWR Analysis
Load the data from the analysis performed from the other notebook.

In [ ]:
#| echo: true
#| 
file_path = "Data/moddat_gwr.csv"

# Load the CSV file
moddatgwr_dfa = pd.read_csv(file_path, low_memory=False)

# Check if the data loaded correctly
#print(moddatgwr_dfa) 

## Summarize Sales Data to find the Average Property in the Study Area

In [ ]:
#| echo: true
# If moddatgwr is still a GeoDataFrame, drop geometry for now

moddatgwr_df = (
    moddatgwr_dfa
      .copy()
      .reset_index(drop=True)
)
if isinstance(moddatgwr_df, gpd.GeoDataFrame):
    moddatgwr_df = moddatgwr_df.drop(columns=moddatgwr_df.geometry.name)

# Add an INTERCEPT column (=1 for every row)
moddatgwr_df["INTERCEPT"] = 1

# Attribute list (keep order identical to X matrix construction)
attributes = [
    "LANDVAL", "GRDFACT", "TLA", "RMOS", "EFFAGE",
    "POOL_AREA", "ATTGAR_AREA", "DETGARAGE_AREA", "TOTPORCH",
    "STHT2", "AGEMAX60", "SEG_OTHER", "CARPRT_AREA", "INTERCEPT"
]

def nz(series):
    """Return series with zeros removed (for min/max/mean/median)."""
    return series.loc[series.ne(0)]

summary_stats = (
    moddatgwr_df[attributes]
      .agg(["count", lambda s: nz(s).min(), lambda s: nz(s).max(),
            lambda s: nz(s).mean(), lambda s: nz(s).median()])
      .T
      .reset_index()
)
summary_stats.columns = ["Variable", "count", "min", "max", "mean", "median"]
summary_stats.head()

## Extract GWR Coefficients

In [ ]:
#| echo: true

coef_cols = [
    "INTERCEPT", "LANDVAL_B", "GRDFACT_B", "TLA_B", "RMOS_B", "EFFAGE_B",
    "POOL_AREA_B", "ATTGAR_AREA_B", "DETGARAGE_AREA_B", "TOTPORCH_B",
    "STHT2_B", "AGEMAX60_B", "SEG_OTHER_B", "CARPRT_AREA_B"
]

gt_coefficients = (
    moddatgwr_df
      .loc[:, ["QUICKREFID", "MASMT"] + coef_cols]
      .head()
)
gt_coefficients

# Location Factor Development
This analysis can be performed on both sales and subjects to get the average property to use for MBV. 

## Build MBV and LOCF

In [ ]:
#| echo: true

# Mapping: feature → column containing its local GWR coefficient
feature_coef = {
    "INTERCEPT"       : "Intercept_B",
    "GRDFACT"         : "GRDFACT_B",
    "RMOS"            : "RMOS_B",
    "LANDVAL"         : "LANDVAL_B",
    "TLA"             : "TLA_B",
    "EFFAGE"          : "EFFAGE_B",
    "POOL_AREA"       : "POOL_AREA_B",
    "ATTGAR_AREA"     : "ATTGAR_AREA_B",
    "DETGARAGE_AREA"  : "DETGARAGE_AREA_B",
    "TOTPORCH"        : "TOTPORCH_B",
    "AGEMAX60"        : "AGEMAX60_B",
    "SEG_OTHER"       : "SEG_OTHER_B",
    "CARPRT_AREA"     : "CARPRT_AREA_B"
}

# ---- mean values from summary_stats ---------------------------
means = (
    summary_stats
      .set_index("Variable")["mean"]
      .to_dict()
)

# ---- contribution columns -------------------------------------
for feat, coef_col in feature_coef.items():
    if feat in means and coef_col in moddatgwr_df.columns:
        moddatgwr_df[f"{feat}_Contribution"] = means[feat] * moddatgwr_df[coef_col]

contrib_cols = [c for c in moddatgwr_df.columns if c.endswith("_Contribution")]

# ---- MBV, MBVavg, LOCF ----------------------------------------
moddatgwr_df["MBVSUM"] = moddatgwr_df[contrib_cols].sum(axis=1)
mbv_avg                 = moddatgwr_df["MBVSUM"].mean()
moddatgwr_df["MBVAVG"]  = mbv_avg
moddatgwr_df["LOCF"]    = moddatgwr_df["MBVSUM"] / mbv_avg

moddatgwr_df.loc[:, ["UNIQUE","QUICKREFID", "LOCF", "MBVSUM"]].head() #+ contrib_cols].head()

# Write to Excel as needed
moddatgwr_df.to_excel("Analysis/moddatgwr_df.xlsx", index=False)

## Map LOCF across the Study Area

In [ ]:
#| label: build-locf-map
#| echo: false
#| warning: false

import folium
import numpy as np
from branca.colormap import linear

# ------------------------------------------------------------------
# Prepare data (as in your previous chunk)
# ------------------------------------------------------------------
plot_df = (
    moddatgwr_df
      .dropna(subset=["LOCF", "COORDLONGX", "COORDLATY"])
      .copy()
)
plot_df["LOCF"] = plot_df["LOCF"].round(4)

lo_min, lo_max = plot_df["LOCF"].min(), plot_df["LOCF"].max()
cmap = linear.RdYlBu_11.scale(lo_min, lo_max).to_step(11)
cmap.caption = "LOCF"

# ------------------------------------------------------------------
# Build map
# ------------------------------------------------------------------
m = folium.Map(
    location=[plot_df["COORDLATY"].median(),
              plot_df["COORDLONGX"].median()],
    zoom_start=10,
    tiles="OpenStreetMap"
)

for _, r in plot_df.iterrows():
    folium.CircleMarker(
        [r["COORDLATY"], r["COORDLONGX"]],
        radius=5,
        color="black", weight=1,
        fill=True, fill_opacity=0.8,
        fill_color=cmap(r["LOCF"]),
        popup=(f"<b>QUICKREFID:</b> {r['QUICKREFID']}<br>"
               f"<b>SEGCLASS:</b> {r['SEGCLASS']}<br>"
               f"<b>LOCF:</b> {r['LOCF']}")
    ).add_to(m)

cmap.add_to(m)
m

# Cluster Analysis 

## Import Clustering Libraries

In [ ]:
#| echo: true
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

## Setup Data For Clustering NBHDS

Create a data frame for how you want to weight your analysis on. In this case SEGCLASS, MASMT, ACTAGE, and TLA

In [ ]:
#| echo: true

#Mapping SEGCLASS to numeric values
segclass_mapping = {
    "R1": 1, 
    "R2": 2, 
    "R3": 3,
    "R4": 4, 
    "R5": 5,
    "R6": 6
}

moddatgwr_df["SEGCLASS_NUM"] = moddatgwr_df["SEGCLASS"].map(segclass_mapping)

# Subset for clustering
clustering_data = moddatgwr_df[["MASMT", "SEGCLASS_NUM", "ACTAGE", "TLA"]].copy()

## Build and Scale Siholuettes

In [ ]:
# Scale the data
scaler = StandardScaler()
clustering_data_scaled = scaler.fit_transform(clustering_data)

# Compute the distance matrix
dist_matrix = squareform(pdist(clustering_data_scaled, metric='euclidean'))

# Perform hierarchical clustering using Ward's method
hc = linkage(clustering_data_scaled, method='ward')

# Plot silhouette scores for optimal cluster selection
silhouette_scores = []
for k in range(2, 11):  # Test cluster sizes from 2 to 10
    clusters = fcluster(hc, k, criterion='maxclust')
    score = silhouette_score(clustering_data_scaled, clusters)
    silhouette_scores.append(score)

plt.figure()
plt.plot(range(2, 11), silhouette_scores, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Method for Optimal Clusters')
plt.show()

## Plot Cluster Dendrograms for Potential Neighrbohood Clusters. 

In [ ]:
plt.figure(figsize=(10, 5))
dendrogram(hc)
plt.title("Hierarchical Clustering Dendrogram")
plt.xlabel("Data Points")
plt.ylabel("Distance")
plt.show()

## Find the optimal number of Clusters

In [ ]:
# Determine the number of clusters for different heights
for h in range(5, 40):
    clusters = fcluster(hc, h, criterion='distance')
    print(f"Height: {h} - Number of clusters: {len(set(clusters))}")

## Replot Dendrogram with Visual Dividers
Dividers at h = 19 (k=13) and h=22 (k=9) 

In [ ]:
plt.figure(figsize=(10, 5))
dendrogram(hc)
plt.title("Hierarchical Clustering Dendrogram")
plt.xlabel("Data Points")
plt.ylabel("Distance")
plt.axhline(y=19, color='red', linestyle='--')  # threshold for clustering
plt.axhline(y=22, color='green', linestyle='--')  #threshold for clustering
plt.show()

## Cut into K Clusters and append to original dataset with summary 
Cutting into 13 clusters (h=19)

In [ ]:
clust_df = (
    moddatgwr_df
      .copy()
)

# Assign clusters to the original dataset
clust_df["nbhd_clust13"] = fcluster(hc, 19, criterion='distance')

# Compute summary statistics by cluster
nbhd_summary = clust_df.groupby("nbhd_clust13").agg(
    n=("LANDVAL", "count"),
    
    MIN_LOCF=("LOCF", "min"), 
    MAX_LOCF=("LOCF", "max"), 
    MEAN_LOCF=("LOCF", "mean"), 
    MED_LOCF=("LOCF", "median"), 
    SD_LOCF=("LOCF", "std"),
    
    MIN_LND=("LANDVAL", "min"), 
    MAX_LND=("LANDVAL", "max"), 
    MEAN_LND=("LANDVAL", "mean"), 
    MED_LND=("LANDVAL", "median"), 
    SD_LND=("LANDVAL", "std"),
   
    MIN_TOTALVAL=("MASMT", "min"), 
    MAX_TOTALVAL=("MASMT", "max"), 
    MEAN_TOTALVAL=("MASMT", "mean"), 
    MED_TOTALVAL=("MASMT", "median"), 
    SD_TOTALVAL=("MASMT", "std"),
    
    MIN_TLA=("TLA", "min"), 
    MAX_TLA=("TLA", "max"), 
    MEAN_TLA=("TLA", "mean"), 
    MED_TLA=("TLA", "median"),
    
    MIN_AGE=("ACTAGE", "min"), 
    MAX_AGE=("ACTAGE", "max"), 
    MEAN_AGE=("ACTAGE", "mean"), 
    MED_AGE=("ACTAGE", "median"),

    MIN_SEGCLASS=("SEGCLASS_NUM", "min"), 
    MAX_SEGCLASS=("SEGCLASS_NUM", "max"), 
    MEAN_SEGCLASS=("SEGCLASS_NUM", "mean"), 
    MED_SEGCLASS=("SEGCLASS_NUM", "median")
).reset_index()

# Display summary statistics
print(nbhd_summary)

# Write to Excel as needed
#nbhd_summary.to_excel("Analysis/nbhd_summary.xlsx", index=False)

## Map New Clusters 

In [ ]:
req_cols = {"nbhd_clust13", "COORDLONGX", "COORDLATY", "QUICKREFID"}
missing  = req_cols - set(clust_df.columns)
if missing:
    raise KeyError(f"Missing columns in clust_df: {missing}")

plot_df = clust_df.dropna(subset=["COORDLONGX", "COORDLATY"]).copy()

# ensure cluster labels are strings (safer for legend keys)
plot_df["nbhd_clust13"] = plot_df["nbhd_clust13"].astype(str)

clusters = sorted(plot_df["nbhd_clust13"].unique())

if len(clusters) != 13:
    raise ValueError(f"Expected 13 clusters but found {len(clusters)}: {clusters}")

# ------------------------------------------------------------------
# 2  Define a 13-colour categorical palette
#     (Set3 only has 12, so we extend with one extra colour)
# ------------------------------------------------------------------
palette_hex = [
    "#8dd3c7", "#ffffb3", "#bebada", "#fb8072", "#80b1d3",
    "#fdb462", "#b3de69", "#fccde5", "#d9d9d9", "#bc80bd",
    "#ccebc5", "#ffed6f", "#1f78b4"          # 13th swatch
]

color_lookup = {cl: palette_hex[i] for i, cl in enumerate(clusters)}

def colour(cl):
    return color_lookup.get(cl, "#666666")

# ------------------------------------------------------------------
# 3  Create Folium map centred on the data
# ------------------------------------------------------------------
center = [plot_df["COORDLATY"].median(), plot_df["COORDLONGX"].median()]
m = folium.Map(location=center, zoom_start=11, tiles="OpenStreetMap")

for _, r in plot_df.iterrows():
    popup = folium.Popup(
        (f"<b>QUICKREFID:</b> {r['QUICKREFID']}<br>"
         f"<b>Cluster ID:</b> {r['nbhd_clust13']}"),
        max_width=250
    )
    folium.CircleMarker(
        [r["COORDLATY"], r["COORDLONGX"]],
        radius=5,
        color="black", weight=1,
        fill=True, fill_opacity=0.85,
        fill_color=colour(r["nbhd_clust13"]),
        popup=popup
    ).add_to(m)

# ------------------------------------------------------------------
# 4  Add a legend
# ------------------------------------------------------------------
legend_html = (
    '<div style="position: fixed; bottom: 50px; left: 50px;'
    ' background: white; z-index: 9999; font-size: 14px;'
    ' border-radius: 8px; padding: 10px; box-shadow: 2px 2px 6px rgba(0,0,0,0.3);">'
    '<b>nbhd_clust13</b><br>'
)
for cl, col in color_lookup.items():
    legend_html += (f'<i style="background:{col}; width:12px; height:12px;'
                    ' display:inline-block; margin-right:5px;"></i>'
                    f'{cl}<br>')
legend_html += '</div>'

m.get_root().html.add_child(folium.Element(legend_html))

# Return map → Quarto injects it inline
m

----------------------------------------------------------------

# Cluster Aggregated NBHDS 
If you already have well defined NBHDS and want to define groupings for Market. We can aggregate LOCF by NBHD and build clusters that way. We can also cluster GWR R2 to see where model is performing well. Also cluster residuals as well. Use your best judgement in building Segments for the Hirearchy. 

## Neighrbohood Summaries

In [ ]:
# Compute summary statistics by cluster
actnbhd_summary = clust_df.groupby("NBHD").agg(
    n=("LANDVAL", "count"),
    
    MIN_LOCF=("LOCF", "min"), 
    MAX_LOCF=("LOCF", "max"), 
    MEAN_LOCF=("LOCF", "mean"), 
    MED_LOCF=("LOCF", "median"), 
    SD_LOCF=("LOCF", "std"),
    
    MIN_LND=("LANDVAL", "min"), 
    MAX_LND=("LANDVAL", "max"), 
    MEAN_LND=("LANDVAL", "mean"), 
    MED_LND=("LANDVAL", "median"), 
    SD_LND=("LANDVAL", "std"),
   
    MIN_TOTALVAL=("MASMT", "min"), 
    MAX_TOTALVAL=("MASMT", "max"), 
    MEAN_TOTALVAL=("MASMT", "mean"), 
    MED_TOTALVAL=("MASMT", "median"), 
    SD_TOTALVAL=("MASMT", "std"),
    
    MIN_R2=("R2", "min"), 
    MAX_R2=("R2", "max"), 
    MEAN_R2=("R2", "mean"), 
    MED_R2=("R2", "median"), 
    SD_R2=("R2", "std")
    
).reset_index()

# Display summary statistics
print(actnbhd_summary)

# Write to Excel as needed
#actnbhd_summary.to_excel("Analysis/actnbhd_summary.xlsx", index=False)

## Subset NBHDS for Clustering 
Can be on median MASMT, median R2, or other

In [ ]:
clustering_data = actnbhd_summary[["MED_TOTALVAL"]].copy()

## Build and Scale Siholuettes

In [ ]:
# Scale the data
scaler = StandardScaler()
clustering_data_scaled = scaler.fit_transform(clustering_data)

# Compute the distance matrix
dist_matrix = squareform(pdist(clustering_data_scaled, metric='euclidean'))

# Perform hierarchical clustering using Ward's method
hc = linkage(clustering_data_scaled, method='ward')

# Plot silhouette scores for optimal cluster selection
silhouette_scores = []
for k in range(2, 11):  # Test cluster sizes from 2 to 10
    clusters = fcluster(hc, k, criterion='maxclust')
    score = silhouette_score(clustering_data_scaled, clusters)
    silhouette_scores.append(score)

plt.figure()
plt.plot(range(2, 11), silhouette_scores, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Method for Optimal Clusters')
plt.show()

## Plot Cluster Dendrograms for Potential NGROUP Clusters. 

In [ ]:
plt.figure(figsize=(10, 5))
dendrogram(hc)
plt.title("Hierarchical Clustering Dendrogram")
plt.xlabel("Data Points")
plt.ylabel("Distance")
plt.show()

## Find the optimal number of NGROUP Clusters 

In [ ]:
# Determine the number of clusters for different heights
for h in range(1, 10):
    clusters = fcluster(hc, h, criterion='distance')
    print(f"Height: {h} - Number of clusters: {len(set(clusters))}")

## Replot Dendrogram with Visual Dividers
Dividers at h = 3 (k=5) and h=6 (k=3) 

In [ ]:
plt.figure(figsize=(10, 5))
dendrogram(hc)
plt.title("Hierarchical Clustering Dendrogram")
plt.xlabel("Data Points")
plt.ylabel("Distance")
plt.axhline(y=3, color='red', linestyle='--')  # threshold for clustering
plt.axhline(y=5, color='green', linestyle='--')  #threshold for clustering
plt.show()

## Cut into K Clusters and assign to NBHD dataset
Cutting into 5 clusters (h=3)

In [ ]:
actclust_df = (
    actnbhd_summary
      .copy()
)

# Assign clusters to the Aggregated dataset
actclust_df["nbhd_clust3"] = fcluster(hc, 3, criterion='distance')

# Join new Clusters to Original Data 

In [ ]:
ngroup_df = (
    moddatgwr_df
        .merge(
            actclust_df[['NBHD', 'nbhd_clust3']],  # limit cols before merge
            on='NBHD',
            how='left'
        )
)

## Summaries on the new NGROUP Clusters

In [ ]:
# Compute summary statistics by cluster
ngroup_summary = ngroup_df.groupby("nbhd_clust3").agg(
    n=("LANDVAL", "count"),
    
    MIN_LOCF=("LOCF", "min"), 
    MAX_LOCF=("LOCF", "max"), 
    MEAN_LOCF=("LOCF", "mean"), 
    MED_LOCF=("LOCF", "median"), 
    SD_LOCF=("LOCF", "std"),
    
    MIN_LND=("LANDVAL", "min"), 
    MAX_LND=("LANDVAL", "max"), 
    MEAN_LND=("LANDVAL", "mean"), 
    MED_LND=("LANDVAL", "median"), 
    SD_LND=("LANDVAL", "std"),
   
    MIN_TOTALVAL=("MASMT", "min"), 
    MAX_TOTALVAL=("MASMT", "max"), 
    MEAN_TOTALVAL=("MASMT", "mean"), 
    MED_TOTALVAL=("MASMT", "median"), 
    SD_TOTALVAL=("MASMT", "std"),
    
    MIN_TLA=("TLA", "min"), 
    MAX_TLA=("TLA", "max"), 
    MEAN_TLA=("TLA", "mean"), 
    MED_TLA=("TLA", "median"),
    
    MIN_AGE=("ACTAGE", "min"), 
    MAX_AGE=("ACTAGE", "max"), 
    MEAN_AGE=("ACTAGE", "mean"), 
    MED_AGE=("ACTAGE", "median"),

    MIN_SEGCLASS=("SEGCLASS_NUM", "min"), 
    MAX_SEGCLASS=("SEGCLASS_NUM", "max"), 
    MEAN_SEGCLASS=("SEGCLASS_NUM", "mean"), 
    MED_SEGCLASS=("SEGCLASS_NUM", "median")
).reset_index()

# Display summary statistics
print(ngroup_summary)

# Write to Excel as needed
#nbhd_summary.to_excel("Analysis/nbhd_summary.xlsx", index=False)

## Map New NGROUP Clusters 

In [ ]:
req_cols = {"nbhd_clust3", "COORDLONGX", "COORDLATY", "QUICKREFID"}
missing  = req_cols - set(ngroup_df.columns)
if missing:
    raise KeyError(f"Missing columns in ngroup_df: {missing}")

plot_df = ngroup_df.dropna(subset=["COORDLONGX", "COORDLATY"]).copy()

cluster_col = "nbhd_clust3"
plot_df[cluster_col] = plot_df[cluster_col].astype(str)

clusters = sorted(plot_df[cluster_col].unique())
if len(clusters) != 5:
    raise ValueError(f"Expected 5 clusters but found {len(clusters)}: {clusters}")

# ----------------------------------------------------------
# 2  Five-colour categorical palette
#    (distinct, colour-blind friendly)
# ----------------------------------------------------------
palette_hex = [
    "#1b9e77",  # teal-green
    "#d95f02",  # orange
    "#7570b3",  # indigo
    "#e7298a",  # magenta-pink
    "#66a61e",  # olive-green
]

color_lookup = {cl: palette_hex[i] for i, cl in enumerate(clusters)}

def colour(cl):
    return color_lookup.get(cl, "#666666")

# ----------------------------------------------------------
# 3  Create Folium map centred on the data
# ----------------------------------------------------------
center = [plot_df["COORDLATY"].median(), plot_df["COORDLONGX"].median()]
m = folium.Map(location=center, zoom_start=11, tiles="OpenStreetMap")

for _, r in plot_df.iterrows():
    popup = folium.Popup(
        f"<b>QUICKREFID:</b> {r['QUICKREFID']}<br><b>Cluster:</b> {r[cluster_col]}",
        max_width=250
    )
    folium.CircleMarker(
        [r["COORDLATY"], r["COORDLONGX"]],
        radius=5,
        color="black", weight=1,
        fill=True, fill_opacity=0.85,
        fill_color=colour(r[cluster_col]),
        popup=popup
    ).add_to(m)

# ----------------------------------------------------------
# 4  Add a legend
# ----------------------------------------------------------
legend_html = (
    '<div style="position: fixed; bottom: 50px; left: 50px;'
    ' background: white; z-index: 9999; font-size: 14px;'
    ' border-radius: 8px; padding: 10px; box-shadow: 2px 2px 6px rgba(0,0,0,0.3);">'
    '<b>nbhd_clust3</b><br>'
)
for cl, col in color_lookup.items():
    legend_html += (f'<i style="background:{col}; width:12px; height:12px;'
                    ' display:inline-block; margin-right:5px;"></i>'
                    f'{cl}<br>')
legend_html += '</div>'

m.get_root().html.add_child(folium.Element(legend_html))

# Return map → Quarto injects it inline
m